In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.astype(int)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [8]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int32

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 

In [17]:
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

In [18]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\sreet\anaconda3\Lib\site-packages\

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [19]:
re=grid.cv_results_

In [20]:
grid_predictions = grid.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [22]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [23]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')

In [24]:
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.8722311775470079


In [25]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [10 39]]


In [26]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.89      0.92      0.90        85
           1       0.85      0.80      0.82        49

    accuracy                           0.87       134
   macro avg       0.87      0.86      0.86       134
weighted avg       0.87      0.87      0.87       134



In [27]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.856782713085234

In [28]:
table=pd.DataFrame.from_dict(re)

In [29]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000695,3.211226e-04,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000962,2.611097e-04,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.003217,1.230263e-03,0.004828,0.001271,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.811321,0.756031,0.832483,0.885265,0.826448,0.042688,7
3,0.001910,5.021896e-04,0.003837,0.000738,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.850809,0.759721,0.886792,0.867097,0.833837,0.045906,6
4,0.001825,4.149289e-04,0.004502,0.000630,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.870047,0.796284,0.870362,0.923510,0.865536,0.040508,1
5,0.002168,4.202079e-04,0.004371,0.001007,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.808927,0.888107,0.775815,0.887907,0.868632,0.845878,0.045479,3
6,0.001200,3.998520e-04,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000277,3.909942e-04,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.001800,3.998995e-04,0.003400,0.000490,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.774691,0.832918,0.813179,0.886792,0.862442,0.834005,0.038854,5
9,0.001800,4.001379e-04,0.003660,0.000763,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.822092,0.850809,0.753180,0.868632,0.903610,0.839665,0.050677,4
